In [1]:
import threading
import time
import queue
from collections import deque
import numpy as np
import pickle
import tensorflow as tf
import pandas as pd
warnings.filterwarnings('ignore')

class RealTimeMouseMonitor:
    def __init__(self, model_path, scaler_path, detection_interval=10):
        self.model_path = model_path
        self.scaler_path = scaler_path
        self.detection_interval = detection_interval
        self.buffer = deque(maxlen=1000)
        self.running = False
        self.anomaly_detected = False
        self.sequence_length = 50
        self.n_features = 6
        self.threshold = 1.0

        # Alert queue for main thread tkinter popups
        self.alert_queue = queue.Queue()

        self.load_model()
        self.setup_mouse_tracking()

    def load_model(self):
        try:
            self.model = tf.keras.models.load_model(
                self.model_path,
                compile=True
            )
            with open(self.model_path.replace('.h5', '_params.pkl'), 'rb') as f:
                params = pickle.load(f)
                self.threshold = params['threshold']
                self.sequence_length = params['sequence_length']
                self.n_features = params['n_features']
            with open(self.scaler_path, 'rb') as f:
                self.scaler = pickle.load(f)
            print("Model, params, scaler loaded successfully!")
        except Exception as e:
            print(f"Error loading model: {e}")

    def setup_mouse_tracking(self):
        try:
            import pynput
            from pynput import mouse
            def on_move(x, y):
                if self.running:
                    timestamp = time.time()
                    self.buffer.append({
                        'x': x,
                        'y': y,
                        'timestamp': timestamp
                    })
            self.mouse_listener = mouse.Listener(on_move=on_move)
            self.mouse_listener.start()
        except ImportError:
            print("Install pynput: pip install pynput")

    def process_buffer_data(self):
        if len(self.buffer) < self.sequence_length:
            return None
        df = pd.DataFrame(list(self.buffer))
        df['dt'] = df['timestamp'].diff().fillna(0)
        df['dx'] = df['x'].diff().fillna(0)
        df['dy'] = df['y'].diff().fillna(0)
        df['distance'] = np.sqrt(df['dx']**2 + df['dy']**2)
        df['vx'] = df['dx'] / (df['dt'] + 1e-8)
        df['vy'] = df['dy'] / (df['dt'] + 1e-8)
        df['speed'] = np.sqrt(df['vx']**2 + df['vy']**2)
        df['ax'] = df['vx'].diff() / (df['dt'] + 1e-8)
        df['ay'] = df['vy'].diff() / (df['dt'] + 1e-8)
        df['acceleration'] = np.sqrt(df['ax']**2 + df['ay']**2)
        df['jerk'] = df['acceleration'].diff() / (df['dt'] + 1e-8)
        df['direction'] = np.arctan2(df['dy'], df['dx'])
        df['direction_change'] = np.abs(df['direction'].diff()).fillna(0)
        df['curvature'] = df['direction_change'] / (df['distance'] + 1e-8)
        feature_columns = ['speed', 'acceleration', 'direction_change', 
                          'curvature', 'jerk', 'dt']
        for col in feature_columns:
            df[col] = df[col].fillna(0)
        if len(df) >= self.sequence_length:
            sequence = df[feature_columns].iloc[-self.sequence_length:].values
            return sequence
        return None

    def detect_anomaly(self, sequence):
        if sequence is None or not hasattr(self, 'scaler'):
            return False, 0.0
        sequence_scaled = self.scaler.transform(sequence)
        sequence_scaled = sequence_scaled.reshape(1, self.sequence_length, self.n_features)
        prediction = self.model.predict(sequence_scaled, verbose=0)
        target = sequence_scaled.reshape(1, -1)
        mse = np.mean((prediction - target) ** 2)
        is_anomaly = mse > self.threshold
        anomaly_score = mse / self.threshold if self.threshold else mse
        return is_anomaly, anomaly_score

    def monitor_loop(self):
        print("Starting real-time mouse monitoring...")
        while self.running:
            try:
                sequence = self.process_buffer_data()
                if sequence is not None:
                    is_anomaly, anomaly_score = self.detect_anomaly(sequence)
                    if is_anomaly and not self.anomaly_detected:
                        print(f"ANOMALY DETECTED! Score: {anomaly_score:.2f}")
                        self.anomaly_detected = True
                        # Instead of showing the alert here, push to the queue!
                        self.alert_queue.put(anomaly_score)
                        threading.Timer(30.0, self.reset_anomaly_flag).start()
                    elif not is_anomaly and self.anomaly_detected:
                        print("Normal pattern resumed")
                        self.anomaly_detected = False
                time.sleep(self.detection_interval)
            except Exception as e:
                print(f"Error in monitoring loop: {e}")
                time.sleep(1)

    def reset_anomaly_flag(self):
        self.anomaly_detected = False

    def start_monitoring(self):
        self.running = True
        self.monitor_thread = threading.Thread(target=self.monitor_loop)
        self.monitor_thread.daemon = True
        self.monitor_thread.start()

    def stop_monitoring(self):
        self.running = False
        if hasattr(self, 'mouse_listener'):
            self.mouse_listener.stop()


In [ ]:
import tkinter as tk
from tkinter import messagebox
import time

if __name__ == "__main__":
    monitor = RealTimeMouseMonitor(
        model_path='mouse_anomaly_model.h5',
        scaler_path='mouse_anomaly_model_scaler.pkl',
        detection_interval=5
    )
    monitor.start_monitoring()
    
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    
    def poll_alert_queue():
        try:
            while True:
                anomaly_score = monitor.alert_queue.get_nowait()
                messagebox.showwarning(
                    "Anomaly Detected",
                    f"Unusual mouse pattern detected!\nAnomaly Score: {anomaly_score:.2f}\n\n"
                    f"This may indicate unauthorized access."
                )
        except queue.Empty:
            pass
        # Call again after 500 ms
        root.after(500, poll_alert_queue)
    
    root.after(500, poll_alert_queue)
    root.mainloop()


Model, params, scaler loaded successfully!
Starting real-time mouse monitoring...
ANOMALY DETECTED! Score: 1.64
Normal pattern resumed
ANOMALY DETECTED! Score: 5.96
Normal pattern resumed
ANOMALY DETECTED! Score: 1.26
Normal pattern resumed
ANOMALY DETECTED! Score: 1.82
Normal pattern resumed
